In [ ]:
# My personal AI which works on voice command and face recognisation to perform basic tasks and behave as an AI chat bot

In [ ]:
# !pip install 

In [48]:
import win32com.client
import speech_recognition as sr
import pyaudio
import wikipedia
import webbrowser
import os
import openai
import time
import numpy 
import requests
import json
import cv2
import face_recognition
import smtplib
import matplotlib.pyplot as plt
from datetime import datetime
from deepface import DeepFace
import mediapipe as mp
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
import numpy as np

query = None   # Global Variables

In [49]:
# todo: to send gmail to anyone

def send_mail(to,content):    

    server = smtplib.SMTP("smtp.gmail.com","587")

    server.ehlo()
    server.starttls()
    server.login("hk.mentor22@gmail.com","cutp fhrd lclp dwyc")
    server.sendmail("hk.mentor22@gmail.com",to,content)
    server.close()

In [50]:
# todo: set the voice for max

speaker = win32com.client.Dispatch("SAPI.SpVoice")

voice_name = speaker.GetVoices()
    
print(voice_name.Item(0).GetAttribute ("Name"))

speaker.Voice

speaker.SetVoice(voice_name.Item(0))

Microsoft David Desktop


In [51]:
# todo: returns text string from voice command

def takecommand():
    
    global query
    
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\nListening.....")
        r.pause_threshold = 1
        audio = r.listen(source)
        
    try:
        print("Recognising.....")
        query = r.recognize_google(audio,language='en-in')
        print(f"\nHrithik: {query}\n")

    except Exception as e:
        print("\nSay that again please....")
    return query

In [52]:
# todo: to get date and time

def time_date():

    timestamp = time.asctime()

    timestamp_list = timestamp.split(" ")

    return timestamp_list

In [53]:
# todo: to get real-time weather report

def get_weather(location):

    param = {
            'q': location,
            'appid': "e087225313f993373f0f426b53895dbb",
            'units': 'imperial'              # to get the temp in F
             }

    weather = requests.get("http://api.openweathermap.org/data/2.5/weather",params=param)

    data = weather.json()

    if weather.status_code == 200:
        #print(data)
        temperature = data['main']['temp']
        description = data['weather'][0]['description']
        humidity = data['main']['humidity']
        wind_speed = data['wind']['speed']

        speaker.Speak(f"Weather report for {location} is as follows")
        print(f'Temperature: {round(((int(temperature)-32)*5)/9, 1)} C')     # converted the temp to C
        speaker.Speak(f'Temperature is {round(((int(temperature)-32)*5)/9, 1)} degree celcius')
        print(f'Description: {description}')
        speaker.Speak(f'Description is {description}')
        print(f'Humidity: {humidity}%')
        speaker.Speak(f'Humidity: {humidity}%')
        print(f'Wind Speed is {wind_speed} m/s')
        speaker.Speak(f'Wind Speed is {wind_speed} metre per second')

In [54]:
def emotion_gesture_detect():
    
    # Load the pre-trained Haar cascade classifier for face detection
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Initialize MediaPipe hands module
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
    mp_drawing = mp.solutions.drawing_utils

    # Initialize pycaw for volume control
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))

    # Define a function to classify hand gestures
    def classify_hand_gesture(landmarks):
        thumb_is_open = landmarks[4].y < landmarks[3].y
        index_is_open = landmarks[8].y < landmarks[6].y
        middle_is_open = landmarks[12].y < landmarks[10].y
        ring_is_open = landmarks[16].y < landmarks[14].y
        pinky_is_open = landmarks[20].y < landmarks[18].y

        current_volume = volume.GetMasterVolumeLevelScalar()

        if thumb_is_open and not index_is_open and not middle_is_open and not ring_is_open and not pinky_is_open:
            new_volume = min(current_volume + 0.0125, 1.0)  # Increase volume
            volume.SetMasterVolumeLevelScalar(new_volume, None)
            return "Thumbs Up"
        elif not thumb_is_open and not index_is_open and not middle_is_open and not ring_is_open and not pinky_is_open:
            new_volume = max(current_volume - 0.0125, 0.0)  # Decrease volume
            volume.SetMasterVolumeLevelScalar(new_volume, None)
            return "Thumb Down"
        return None

    # Initialize the camera
    video = cv2.VideoCapture(0)

    # Custom thresholds for each emotion
    custom_thresholds = {
        'angry': 0.2,
        'disgust': 0.05,
        'fear': 0.4,
        'happy': 0.6,
        'sad': 0.25,
        'surprise': 0.35,
        'neutral': 0.4
    }

    while True:
        __, frame = video.read()
        
        # Find all face locations in the current frame
        face_locations = face_recognition.face_locations(frame)

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the grayscale frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

        # Process the image and find hands
        results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Draw hand landmarks and classify gestures
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                gesture = classify_hand_gesture(hand_landmarks.landmark)
                if gesture:
                    cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Process each detected face
        for (x, y, w, h) in faces:
            # Extract the face region from the grayscale frame
            face_roi = gray[y:y+h, x:x+w]

            try:
                # Convert the face region to RGB format (DeepFace requires RGB input)
                face_rgb = cv2.cvtColor(face_roi, cv2.COLOR_GRAY2RGB)

                # Perform emotion analysis on the face region
                analyze = DeepFace.analyze(face_rgb, actions=['emotion'], enforce_detection=False)
                
                #Check if the result is as expected
                emotions = analyze[0]
                #print("Dominant Emotion:", emotions['dominant_emotion'])
                print('Face Confidence:',emotions['face_confidence'])
                
                
                # Display the filtered dominant emotions on the frame    
                dominant_emotion_text = emotions['dominant_emotion']
                text_x = frame.shape[1] - 10 - len(dominant_emotion_text) * 20  # Adjusted position
                text_y = 30  # Adjusted position
                cv2.putText(frame, dominant_emotion_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0,255), 2)
    
                # Draw a rectangle around the detected face
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            except Exception as e:
                print('Error analyzing face:', e)

        # Display the resulting frame with detected faces and emotions
        cv2.imshow('Face Emotion and Gesture Detection', frame)

        # Check for exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    video.release()
    cv2.destroyAllWindows()


In [55]:
def face_detect():
    
    # Load a sample image of yourself
    your_image = face_recognition.load_image_file("C:/Users/asus/OneDrive/Desktop/WhatsApp Image 2024-03-06 at 12.42.55_07df780f.jpg")
    your_encoding = face_recognition.face_encodings(your_image)[0]

    # Initialize the camera
    video = cv2.VideoCapture(0)

    while True:
        __, frame = video.read()
        
        # Find all face locations in the current frame
        face_locations = face_recognition.face_locations(frame)

        # If faces are found, try to recognize them
        if face_locations:
            face_encodings = face_recognition.face_encodings(frame, face_locations)
            for face_encoding in face_encodings:
                # Compare the current face with the known face
                match = face_recognition.compare_faces([your_encoding], face_encoding)
                
                if match[0]==True:
                    return match[0]
                
                else:
                    return match[0]

            
        # Check for exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    video.release()
    cv2.destroyAllWindows()    

In [56]:
def main():
    
    global query
    
    if face_detect():
        
        print('Face Recognised')
        emotion_gesture_detect()
        
        # dictionary to store esentinal apps name and location on PC
        app_dic = {'code':"C:/Program Files/Microsoft Visual Studio/2022/Community/Common7/IDE/devenv.exe",
                'chrome':"C:/Program Files/Google/Chrome/Application/chrome.exe",
                'excel':"C:/Program Files/Microsoft Office/root/Office16/EXCEL.EXE",
                'word':"C:/Program Files/Microsoft Office/root/Office16/WINWORD.EXE",
                'powerpoint':"C:/Program Files/Microsoft Office/root/Office16/POWERPNT.EXE",
                'my pc':"C:/Users/asus/Desktop/My PC - Shortcut.lnk",
                'edge':"C:/Program Files (x86)/Microsoft/Edge/Application/msedge.exe"}   
        
        get_time = time_date() 
        time = get_time[3].split(":")                     # to get through date and time function

        if (int(time[0])>=5 and int(time[0])<12):
            speaker.Speak("\nGood morning! Hrithik")

        elif (int(time[0])>=12 and int(time[0])<18):
            speaker.Speak("\nGood afternoon! Hrithik")

        elif (int(time[0])>18):
            speaker.Speak("\nGood evening! Hrithik")
        
        while True:

            query = takecommand().lower()
            #query =  input("Enter your command: ").lower()
            print(query)

            if 'send mail' in query:
                speaker.Speak("Please tell me to whom you want to send?")
                #to = takecommand().lower()
                to = input("To: ")
                speaker.Speak("Now please tell me what you want to send?")
                #content = takecommand().lower()
                content = input("Your Mail: ")
                
                try:
                    send_mail(to,content)
                    speaker.Speak("Mail has been send!")

                except exception as e:
                    print(e)
                    speaker.Speak("Sorry i am unable to send your mail right now")
                
            elif "open youtube" in query:
                speaker.Speak("Opening YouTube")
                webbrowser.open("youtube.com")
                
            elif 'good bye' in query or 'goodbye' in query:
                speaker.Speak("Good bye Hrithik! Have a nice day")
                break

            elif "google chrome" in query:
                speaker.Speak("Opening Google Chrome")
                os.startfile(f"{app_dic['chrome']}")   
            
            elif "date" in query:
                speaker.Speak(f"Today is {int(get_time[2])} {get_time[1]} {get_time[4]} {get_time[0]} ")
                
            elif "time" in query:
                time_24hr_format = "%H:%M:%S"
                
                datetime_obj = datetime.strptime(time_24hr, time_24hr_format)
                
                time_12hr_format = datetime_obj.strftime("%I:%M:%S %p")
                
                speaker.Speak(time_12hr_format)

            elif "hello max" in query:

                print("\nHello Hrithik\nMax here!")

                while "bye max" not in query or 'byemax' not in query :

                    #query  = input("Enter your command: ")
                    query = takecommand().lower()

                    openai.api_key = "sk-proj-vXoizBV4V0cnhqilD1PmT3BlbkFJlsljbj0ue1PNzVx2pGfZ"

                    chat_completion = openai.chat.completions.create(
                        messages=[
                            {
                                "role": "user",
                                "content": query,
                            }
                        ],
                        temperature=1,
                        max_tokens=4000,
                        top_p=0.5,
                        frequency_penalty=1,
                        presence_penalty=1,
                        model="gpt-3.5-turbo",
                    )

                    print(f"Max: {chat_completion.choices[0].message.content}")
                    speaker.Speak(chat_completion.choices[0].message.content)

            elif "code" in query:
                speaker.Speak("Opening VS Code")
                os.startfile(f"{app_dic['code']}")

            elif "excel" in query:
                speaker.Speak("Opening MS Excel")
                os.startfile(f"{app_dic['excel']}")

            elif "ms word" in query:
                speaker.Speak("Opening MS Word")
                os.startfile(f"{app_dic['word']}")

            elif "powerpoint" in query:
                speaker.Speak("Opening MS Powerpoint")
                os.startfile(f"{app_dic['powerpoint']}")

            elif 'my pc' in query:
                speaker.Speak("Opening My PC")
                os.startfile(f"{app_dic['my pc']}")

            elif 'edge' in query:
                speaker.Speak("Opening Micosoft Edge")
                os.startfile(f"{app_dic['edge']}")

            elif 'weather' in query:
                speaker.Speak("For which location?")
                get_weather(takecommand())
                #get_weather(input("Location:"))
                
            else:
                speaker.Speak("Please! try again")
    else:
        print("Sorry! Face not recognised")
        speaker.Speak("Sorry! face not recognised")
        

In [ ]:
if __name__ == "__main__":
    main()

Face Recognised
Face Confidence: 0.94
Face Confidence: 0.9
Face Confidence: 0.91
Face Confidence: 0.89
Face Confidence: 0.91
Face Confidence: 0.91
Face Confidence: 0.9
Face Confidence: 0.91
Face Confidence: 0.92

Listening.....
Recognising.....

Hrithik: hey Max what is the date today

hey max what is the date today

Listening.....
Recognising.....

Hrithik: hey Max what is the date today

hey max what is the date today

Listening.....
Recognising.....

Hrithik: amax what is the date today

amax what is the date today

Listening.....
Recognising.....

Hrithik: hey Max what is the time

hey max what is the time

Listening.....
Recognising.....

Hrithik: hey Max what is the date today

hey max what is the date today

Listening.....
Recognising.....
